In [ ]:
# 📌 Ячейка 1: настройки
import os, re, json, requests, glob

OPENROUTER_API_KEY = ""

GRAPH_FILE = "graphcontent.js"
INPUT_DIR = "./"
OUTPUT_FILE = "graphcontent_updated.js"

# Проверим, что исходник есть
if not os.path.exists(GRAPH_FILE):
    raise FileNotFoundError(f"{GRAPH_FILE} не найден в {os.getcwd()}")
print("✅ graphcontent.js найден")


✅ graphcontent.js найден


In [21]:
import ast

# 📌 Ячейка 2: читаем rawNodes из JS
with open(GRAPH_FILE, "r", encoding="utf-8") as f:
    js_text = f.read()

# Найдём массив максимально "ленивым" матчем (non-greedy)
match = re.search(r"export const rawNodes\s*=\s*(\[.*?\]);", js_text, re.S)
if not match:
    raise ValueError("Не удалось найти rawNodes в файле")

raw_nodes_str = match.group(1)

# Иногда в JS могут быть комментарии — уберём // ... и /* ... */
raw_nodes_str = re.sub(r"//.*", "", raw_nodes_str)
raw_nodes_str = re.sub(r"/\*.*?\*/", "", raw_nodes_str, flags=re.S)

# Теперь загружаем
raw_nodes = json.loads(raw_nodes_str)
print(f"🔎 Загружено {len(raw_nodes)} узлов")


🔎 Загружено 163 узлов


In [22]:
# 📌 Ячейка 3: функция для нейронки
API_URL = "https://openrouter.ai/api/v1/chat/completions"
HEADERS = {"Authorization": f"Bearer {OPENROUTER_API_KEY}", "Content-Type": "application/json"}

SYSTEM_PROMPT = (
    "Ты умный ассистент, который делает очень краткие конспекты. "
    "Тебе даётся LaTeX-файл с учебным материалом. "
    "Сделай суперкороткий пересказ на русском (2–3 предложения), как в тезисах. "
    "Не пиши никаких вступлений, не используй слова вроде 'рассматривается', 'описывается', "
    "'в этом файле', 'конспект'. "
    "Сразу пиши понятия и темы которые рассматриваются в конспекте, не используй сложных .tex команд"
)

def describe_tex_file(filepath: str) -> str:
    with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
        content = f.read()

    payload = {
        "model": "anthropic/claude-3.5-sonnet",  # или gpt-4o-mini
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": content[:8000]}
        ],
        "max_tokens": 300,
        "temperature": 0.2,  # пониже, чтобы было сухо и точно
    }

    resp = requests.post(API_URL, headers=HEADERS, data=json.dumps(payload), timeout=90)
    if resp.status_code != 200:
        raise RuntimeError(f"API error {resp.status_code}: {resp.text[:200]}")
    j = resp.json()
    try:
        return j["choices"][0]["message"]["content"].strip()
    except KeyError:
        return j["choices"][0].get("text", "").strip()


In [23]:
# 📌 Ячейка 4: собираем описания для всех .tex
tex_files = sorted(glob.glob(os.path.join(INPUT_DIR, "*.tex")))
print(f"Найдено {len(tex_files)} .tex файлов")

# Словарь number -> summary
summaries = {}
for path in tex_files:
    try:
        number = int(os.path.splitext(os.path.basename(path))[0])
    except ValueError:
        continue
    print(f"Обработка {number}.tex ...")
    try:
        summary = describe_tex_file(path)
        summaries[number] = summary
        print(f"  ✅ {summary[:80]}...")
    except Exception as e:
        print(f"  ❌ ошибка для {number}.tex: {e}")


Найдено 85 .tex файлов
Обработка 10.tex ...
  ✅ Закон Гука описывает силу упругости пружины, пропорциональную деформации и напра...
Обработка 100.tex ...
  ✅ Метод расчёта сопротивления бесконечной электрической цепи с повторяющимися элем...
Обработка 101.tex ...
  ✅ Источники ЭДС имеют внутреннее сопротивление r и поддерживают разность потенциал...
Обработка 102.tex ...
  ✅ Любую электрическую схему с источниками и резисторами можно заменить одним эквив...
Обработка 103.tex ...
  ✅ Амперметр включается последовательно в цепь для измерения силы тока и имеет мало...
Обработка 104.tex ...
  ✅ Вольт-амперная характеристика (ВАХ) показывает зависимость тока от напряжения в ...
Обработка 105.tex ...
  ✅ Мостовая схема с пятью сопротивлениями сбалансирована при условии R₂R₃ = R₁R₄, к...
Обработка 106.tex ...
  ✅ Трёхвыводную электрическую сеть можно представить в виде эквивалентной "звезды" ...
Обработка 11.tex ...
  ✅ Сила - мера взаимодействия тел, измеряемая в ньютонах и характеризующаяся

In [24]:
# 📌 Ячейка 5: обновляем rawNodes
updated_nodes = []
for node in raw_nodes:
    data = node.get("data", {})
    number = data.get("number")
    if number in summaries:
        data["text"] = summaries[number]  # перезаписываем или добавляем
    node["data"] = data
    updated_nodes.append(node)

# Собираем обратно в JS
updated_js = "export const rawNodes = " + json.dumps(updated_nodes, ensure_ascii=False, indent=2) + ";"

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    f.write(updated_js)

print(f"🎉 Обновленный файл записан в {OUTPUT_FILE}")


🎉 Обновленный файл записан в graphcontent_updated.js
